In [41]:
import boto3
from botocore.config import Config

region_name = "us-west-2"
#high_level_model = "anthropic.claude-3-5-sonnet-20240620-v1:0"
#high_level_model = "anthropic.claude-3-sonnet-20240229-v1:0"
high_level_model = "anthropic.claude-3-haiku-20240307-v1:0"
low_level_model = "anthropic.claude-3-haiku-20240307-v1:0"

def converse_with_bedrock(sys_prompt, usr_prompt, model_id):
    temperature = 0
    top_p = 0.1
    top_k = 1
    inference_config = {"temperature": temperature, "topP": top_p}
    additional_model_fields = {"top_k": top_k}
    response = boto3_client.converse(
        modelId=model_id, 
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
    return response['output']['message']['content'][0]['text']

def init_boto3_client(region: str):
    retry_config = Config(
        region_name=region,
        retries={"max_attempts": 10, "mode": "standard"}
    )
    return boto3.client("bedrock-runtime", region_name=region, config=retry_config)


def create_prompt(sys_template, user_template, **kwargs):
    sys_prompt = [{"text": sys_template.format(**kwargs)}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template.format(**kwargs)}]}]
    return sys_prompt, usr_prompt

boto3_client = init_boto3_client(region_name)


In [3]:
from py2neo import Graph
import os

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Graph()

In [4]:
def select_subgraph_dev(question, graph):
    question = question
    query = """
        MATCH (n:Title {level: "1"})
        RETURN n.value, id(n) as node_id
    """
    results = graph.run(query)
    subgraph_list = [(record["n.value"], record["node_id"]) for record in results]
    subgraph_list_with_number = [f"{i}. {subgraph[0]}" for i, subgraph in enumerate(subgraph_list)]

    sys_prompt_template = """ 
    You are an expert engineer well-versed in AWS manual documents. 
    Your task is to select the most appropriate manual document name for the user's question. 
    If there are no relevant documents, provide an empty list (""). """

    usr_prompt_template = """ 
    Please select the single most relevant document name for the given question.

    #Question: {question}

    #Document List: {subgraph_list_with_number}

    #Response Format: Provide only the index number of the selected document (omit any preamble) """
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, subgraph_list_with_number=subgraph_list_with_number)
    
    model_id = low_level_model 
    selected_id = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
    try:
        if selected_id == "":
            return [], "", "generate_answer"

        else: 
            selected_subgraph_id = subgraph_list[int(selected_id)][1]
            print("Selected:", subgraph_list[int(selected_id)][0])
            return [selected_subgraph_id], subgraph_list[int(selected_id)][0], "traverse_child"
    except:
        return [], "", "generate_answer"

In [5]:
# 질문의 주제 선택
question = "Bedrock에서 제공하는 모델 목록"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child


In [6]:
# 질문의 주제 선택 (현재 없는 내용)
question = "SageMaker에서 모델을 배포하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

[] |  | generate_answer


In [7]:
class TraverseResult:
    def __init__(self, parent_id, parent_name, child_level, selected_child_ids, child_names, next_action):
        self.parent_id = parent_id
        self.parent_name = parent_name
        self.child_level = child_level
        self.selected_child_ids = selected_child_ids
        self.child_names = child_names
        self.next_action = next_action

In [8]:
csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def traverse_child_dev(question, subgraph, graph, target_node):
    parent_id = target_node[0]
    query = """
        MATCH (n)
        WHERE id(n) = $parent_id
        OPTIONAL MATCH (n)-[:HAS_CHILD]->(c)
        RETURN n.value as parent_name, c.level as child_level, c.value as child_name, id(c) as child_id
    """
    params = {"parent_id": parent_id}
    query_results = graph.run(query, params)

    parent_name = None
    child_level = None
    child_list = []
    child_names = []

    for record in query_results:
        if parent_name is None:
            parent_name = record["parent_name"]
        if child_level is None:
            child_level = record["child_level"]
        if record["child_name"] is not None:
            child_list.append((record["child_name"], record["child_id"]))
            child_names.append(record["child_name"])

    print(f"Traversing '{parent_name}'...")

    if not child_list:
        print("No child. Proceed to 'get_contents'...")
        #print(f"Debug: {parent_id}, {parent_name}, {child_level}, [], [], 'get_contents'")
        return TraverseResult(parent_id, parent_name, -1, [], [], "get_contents")

    child_list_with_number = [f"{i}. {child}" for i, child in enumerate(child_list)]
    sys_prompt_template = """
    당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.
    당신의 임무는 사용자의 질문에 답변하기 위해, <{subgraph}> 매뉴얼 문서에서 가장 관련성 높은 하위 메뉴를 선택하는 것입니다.

    작업 순서:
    1. 주어진 하위 메뉴 목록을 검토하여 직접적으로 연관된 메뉴들을 찾습니다.
    2. 연관성이 가장 높은 메뉴를 선택하여, 인덱스 번호(0부터 시작)로 응답합니다.
    3. 질문과 매우 밀접한 메뉴가 1개 이상인 경우, 선택한 메뉴의 인덱스 번호 목록으로 응답합니다.

    선택 기준:
    - 질문의 핵심 키워드와 일치하고 질문의 맥락에 맞는 메뉴를 우선적으로 고려하세요.
    - 일반적 가이드보다는 질문의 특정 주제나 기능을 다루는 메뉴를 선호합니다. 예를 들어, 'Getting started' 가이드보다는 특정 기능이나 서비스에 대한 상세 설명이 있는 항목을 선호합니다.
    - 반드시 선택을 해야하는 것은 아닙니다. 연관성이 낮거나 불확실한 메뉴는 선택하지 마세요.

    """
    usr_prompt_template = """
    #질문: {question}

    #메뉴 목록:
    {child_list_with_number}

    #응답 형식: {csv_list_response_format}.
    """


    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, subgraph=subgraph, question=question, child_list_with_number=child_list_with_number, csv_list_response_format=csv_list_response_format)
    model_id = high_level_model
    selected_ids = converse_with_bedrock(sys_prompt, usr_prompt, model_id)

    try:
        selected_id_list = [int(id.strip()) for id in selected_ids.split(',') if id.strip().isdigit()]

        if not selected_id_list:
            #print(f"Debug1: {parent_id}, {parent_name}, {child_level}, {selected_child_ids}, {selected_child_names}, 'traverse_child'")
            return TraverseResult(parent_id, parent_name, -1, [], [], "get_contents")
        
        selected_child_ids = [child_list[id][1] for id in selected_id_list if id < len(child_list)]
        selected_child_names = [child_list[id][0] for id in selected_id_list if id < len(child_list)]

        #print(f"Debug2: {parent_id}, {parent_name}, {child_level}, {selected_child_ids}, {selected_child_names}, 'traverse_child'")
        return TraverseResult(parent_id, parent_name, child_level, selected_child_ids, selected_child_names, "traverse_child")
    
    except Exception as e:
        #print(f"Debug3: Exception occurred: {str(e)}")
        #print(f"Debug4: {parent_id}, {parent_name}, {child_level}, {selected_child_ids}, {selected_child_names}, 'traverse_child'")
        return TraverseResult(parent_id, parent_name, -1, [], [], "get_contents")

In [9]:
class Context:
    def __init__(self, parent_id, parent_name, contents, contents_length, search_type, k):
        self.parent_id = parent_id
        self.parent_name = parent_name
        self.contents = contents
        self.contents_length = contents_length
        self.search_type = search_type
        self.k = k

In [33]:
traverse_results = []

# 질문의 주제 선택
question = "Bedrock에서 Custom Model 활용 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1908]
Traversing 'Custom models'...
target_node: [2023]
Traversing 'Use a custom model'...
No child. Proceed to 'get_contents'...


In [11]:
def get_contents_dev(graph, parent_id, k=5):
    count_query = """
        MATCH (n)-[:HAS_CONTENTS]->(c)
        WHERE id(n) = $parent_id
        RETURN count(c) as contents_length, n.value as parent_name
    """
    params = {"parent_id": parent_id}
    count_result = graph.run(count_query, params).data()[0]
    contents_length = count_result['contents_length']
    parent_name = count_result['parent_name']
    print(f"Num Documents: {contents_length}")

    if contents_length <= k * 2:
        search_type = "get_short_documents"
        content_query = """
            MATCH (n)-[:HAS_CONTENTS]->(c)
            WHERE id(n) = $parent_id
            RETURN c.text
            ORDER BY c.order
            LIMIT $k
        """
        params = {"parent_id": parent_id, "k": k}
        content_results = graph.run(content_query, params)
        contents = [record["c.text"] for record in content_results]
        context = " ".join(contents)

    else:
        search_type = "node_level_search"
        context = ""

    return Context(parent_id, parent_name, context, contents_length, search_type, k)

In [12]:
# 질문의 주제 선택
question = "Bedrock에서 활용가능한 모델 목록"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
print("==============")
print(context.contents)
print("==============")

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [116]
Traversing 'Supported foundation models in Amazon Bedrock'...
target_node: [134]
Traversing 'Model support by feature'...
No child. Proceed to 'get_contents'...
Num Documents: 6
Search Type: get_short_documents
Note You can run inference on all available FMs. The following table details the support for features that are limited to certain FMs. Model support by feature 45 |Model|Model evaluati n|Knowled o base (embed gs)|gKen owled base din(q uery)|gAeg ents|Fine- tuning (custom models)|Continu pre- train ing (custom models)|edP rovisio ed Through t|n Tool use pu|Convers API| |---|---|---|---|---|---|---|---|---|---| |AI21 Jamba- Ins truct|No|N/A|No|No|No|No|No|No|Yes| |Amazon Titan Text G1 - Express|Yes|N/A|No|No|Yes|Yes|Yes|No|Yes| |Amazon Titan Text G1 - Lite|Yes|N/A|No|No|Yes|Yes|Yes|No|Yes| |Amazon Titan Text Premier|No|N/A|Yes|Yes|Yes (preview|No )|Yes 

In [29]:
# 질문의 주제 선택
question = "Bedrock의 Agent에서 SDK를 활용해서 Agent Action을 정의하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1359]
Traversing 'Agents for Amazon Bedrock'...
target_node: [1578]
Traversing 'Customize an Amazon Bedrock agent'...
target_node: [1581]
Traversing 'Advanced prompts in Amazon Bedrock'...
target_node: [1606]
Traversing 'Parser Lambda function in Agents for Amazon Bedrock'...
No child. Proceed to 'get_contents'...
Num Documents: 82
Search Type: node_level_search


In [14]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

searching_scheme = "full_text" # full_text | keyword | vector
csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def node_level_search_dev(question, graph, parent_id, parent_name, language = "English", k=5):

    if searching_scheme == "vector":
        sys_prompt_template = """
            당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 벡터 검색으로 문서를 찾아내기에 적합한 질문을 만들어주세요.({language})
            
            주의: 

            - 문서 이름과 질문을 고려하여 해당 문서 내에서 가장 관련성 높고 특징적인 질문을 생성하세요.
            - 문서의 특정 내용을 잘 나타내는 자연어 질문을 선호합니다.
        """
        usr_prompt_template = "#검색 대상 문서 이름:\n{subgraph}\n\n#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"

        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, subgraph=subgraph, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)

        index_name = "content_embedding_index"
        embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name=region_name)
        vector_store = Neo4jVector.from_existing_index(
            embedding=embeddings,
            index_name=index_name,
            node_label="Content",
            text_node_property="text", 
            embedding_node_property="embedding"
        )

        question_embedding = embeddings.embed_query(keywords)

        vector_search_query = """
            MATCH (parent)-[:HAS_CONTENTS]->(child:Content)
            WHERE id(parent) = $parent_id
            WITH child
            CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
            WHERE node = child
            RETURN id(node) AS node_id, node.text AS text, score
            ORDER BY score DESC
            LIMIT $k
        """

        params = {
            "parent_id": parent_id,
            "question_embedding": question_embedding,
            "k": k,
            "index_name": index_name
        }

        search_results = vector_store.query(vector_search_query, params=params)
        
    else:
        sys_prompt_template = """
        당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 핵심 키워드를 1개 추출합니다.
        키워드는 다음 조건을 반드시 만족해야 합니다:
        1. 키워드에 '_', '-' 등의 특수 문자 포함 금지 (예: custom_model 대신 custom model로 응답)
        2. 주어진 문서 이름 내에서 질문의 맥락에 가장 적합한 단어를 선택 (문서 이름은 키워드에 포함할 필요가 없음)
        2. 문서 이름에 이미 포함된 내용보다는 검색하려는 특정 기능 및 개념을 잘 나타내는 구체적 단어를 선택
        3. {language} 키워드 제공 
        
        주의: 
        - 문서 이름과 질문을 고려하여 해당 문서 내에서 가장 관련성 높고 특징적인 단어를 선택하세요.
        - 너무 일반적인 단어보다는 문서의 특정 내용을 잘 나타내는 단어를 선호합니다.
        """
        usr_prompt_template = "#검색 대상 문서 이름:\n{parent_name}\n\n#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"

        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, parent_name=parent_name, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
    

        if searching_scheme == "full_text":
            search_query = """MATCH (parent)-[:HAS_CONTENTS]->(child)
                WHERE id(parent) = $parent_id
                WITH child
                CALL db.index.fulltext.queryNodes("Search_Content_by_FullText", $keywords) YIELD node, score
                WHERE node = child
                RETURN node.text as text, score
                ORDER BY score DESC
                LIMIT $k
            """
        elif searching_scheme == "keyword":
            search_query = """MATCH (parent)-[:HAS_CONTENTS]->(child)
                WHERE id(parent) = $parent_id
                WITH child, $keywords AS keyword
                WHERE child.text CONTAINS keyword
                RETURN child.text AS text, 
                    size(split(toLower(child.text), toLower(keyword))) - 1 AS score
                ORDER BY score DESC
                LIMIT $k
            """
        
        params = {"parent_id": parent_id, "keywords": keywords, "k": k}
        search_results = graph.run(search_query, params)
        
    content = "\n\n\n".join(f"{record['text']} (Score: {record['score']})" for record in search_results)
    return content

In [34]:
# 질문의 주제 선택
question = "Bedrock의 Agent에서 SDK를 활용해서 Agent Action을 정의하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    search_content = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)
    print(search_content)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1359]
Traversing 'Agents for Amazon Bedrock'...
target_node: [1393]
Traversing 'Create an action group for an Amazon Bedrock agent'...
target_node: [1396]
Traversing 'Defining actions in the action group'...
target_node: [1404]
Traversing 'Define OpenAPI schemas for your agent's action groups in Amazon Bedrock'...
No child. Proceed to 'get_contents'...
Num Documents: 13
Search Type: node_level_search



In [16]:
def check_relevance_dev(question, content, parent_name, contents_length, search_type, k=5):
    optional_prompt1 = ""
    optional_prompt2 = ""
    
    if search_type == "get_short_documents" and contents_length > k:
        optional_prompt1 = "- 사전 정보가 질문 취지에 부합하지만 뒷 내용 추가 확인 필요: 'Partial'"
        optional_prompt2 = "또는 `Partial`"

    sys_prompt_template = """
    당신은 유능한 데이터 분석가입니다. 당신의 임무는 오직 주어진 사전 정보만을 활용하여 질문에 답변 가능한지, 아래의 기준으로 판단하는 것입니다.
    
    판단 기준:
    1. 질문의 핵심 키워드가 문서 이름 또는 사전 정보에 등장하는가?
    2. 알아내고자 하는 구체적 정보가 포함되어 있는가?

    응답 방법:
    - 문서 이름 및 사전 정보가 질문과 관련 없음: 'None'
    - 사전 정보만으로 질문에 답변 가능: 'Complete'
    {partial1}

    서두는 생략하고, `None` 또는 `Complete`{partial2}으로만 답변하세요.
    """
    usr_prompt_template = """
    #사전 정보 (문서이름: {parent_name})
    {context}
    
    #질문: {question}
    """

    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, partial1=optional_prompt1, partial2=optional_prompt2, parent_name=parent_name, question=question, context=content)
    model_id = high_level_model
    status = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
    
    return status

In [17]:
# 질문의 주제 선택
question = "model customization 작업에서 데이터 암호화하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    search_content = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)
    print("==============")
    print(search_content)
    print("==============")
    status = check_relevance_dev(question, search_content, context.parent_name, context.contents_length, context.search_type, context.k)
else:
    print("==============")
    print(context.contents)
    print("==============")
    status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1908]
Traversing 'Custom models'...
target_node: [1944]
Traversing 'Submit a model customization job'...
No child. Proceed to 'get_contents'...
Num Documents: 6
Search Type: get_short_documents
You can create a custom model by using Fine-tuning or Continued Pre-training in the Amazon Bedrock console or API. The customization job can take several hours. The duration of the job depends on the size of the training data (number of records, input tokens, and output tokens), number of epochs, and batch size. Select the tab corresponding to your method of choice and follow the steps. Console To submit a model customization job in the console, carry out the following steps. 1. Sign in to the AWS Management Console using an IAM role with Amazon Bedrock [permissions, and open the Amazon Bedrock console at https://console.aws.amazon.com/](https://console.aws.amazon.com/bedr

In [35]:
# 질문의 주제 선택
question = "Bedrock의 Custom Model 활용 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    search_content = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)
    print("==============")
    print(search_content)
    print("==============")
    status = check_relevance_dev(question, search_content, context.parent_name, context.contents_length, context.search_type, context.k)
else:
    print("==============")
    print(context.contents)
    print("==============")
    status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1908]
Traversing 'Custom models'...
target_node: [2023]
Traversing 'Use a custom model'...
No child. Proceed to 'get_contents'...
Num Documents: 3
Search Type: get_short_documents
Before you can use a customized model, you need to purchase Provisioned Throughput for it. For more information about Provisioned Throughput, see Provisioned Throughput for Amazon Bedrock. You can then use the resulting provisioned model for inference. Select the tab corresponding to your method of choice and follow the steps. Console To purchase Provisioned Throughput for a custom model. 1. Sign in to the AWS Management Console using an IAM role with Amazon Bedrock [permissions, and open the Amazon Bedrock console at https://console.aws.amazon.com/](https://console.aws.amazon.com/bedrock/) [bedrock/.](https://console.aws.amazon.com/bedrock/) 2. From the left navigation pane, choose Cus

In [19]:
# 질문의 주제 선택
question = "Bedrock Agent에서 memory 기능을 활용하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("context:", context.parent_name)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    search_content = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)
    print("==============")
    print(search_content)
    print("==============")
    status = check_relevance_dev(question, search_content, context.parent_name, context.contents_length, context.search_type, context.k)
else:
    print("==============")
    print(context.contents)
    print("==============")
    status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1359]
Traversing 'Agents for Amazon Bedrock'...
target_node: [1454]
Traversing 'Use memory to retain conversational context across multiple sessions'...
target_node: [1458]
Traversing 'Configure memory for your Amazon Bedrock agent'...
No child. Proceed to 'get_contents'...
Num Documents: 4
context: Configure memory for your Amazon Bedrock agent
Search Type: get_short_documents
To configure memory for your agent, you must first enable memory and then optionally specify [the retention period for the memory. You can enable memory for your agent when you create or](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create.html) [update your agent.](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-manage.html#agents-edit) To learn how to configure memory for your agent, select the tab corresponding to your method of choice and follow steps. Consol

In [20]:
def get_sibling_contents_dev(graph, parent_id, content, k=5):
    content_query = """
        MATCH (n)-[:HAS_CONTENTS]->(c)
        WHERE id(n) = $parent_id AND c.order >= $order_pos
        RETURN c.text
        ORDER BY c.order
        LIMIT $k
    """       
    trial = 1
    order_pos = k * trial
    params = {"parent_id": parent_id, "k": k, "order_pos": order_pos} 

    content_results = graph.run(content_query, params)
    sibling_content = [record["c.text"] for record in content_results]
    contents = " ".join([content] + sibling_content)

    return contents

In [36]:
# 질문의 주제 선택
question = "Bedrock에서 Knowledgebase의 소스로 웹 페이지를 사용하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

print("==============")
print(context.contents)
print("==============")
status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1070]
Traversing 'Knowledge bases for Amazon Bedrock'...
target_node: [1175]
Traversing 'Data source connectors'...
target_node: [1258]
Traversing 'Crawl web pages for your Amazon Bedrock knowledge base'...
Num Documents: 2
context: Crawl web pages for your Amazon Bedrock knowledge base
Search Type: get_short_documents
Note Crawling web URLs as your data source is in preview release and is subject to change. The Amazon Bedrock provided Web Crawler connects to and crawls URLs you have selected for use in your Amazon Bedrock knowledge base. You can crawl website pages in accordance with [your set scope or limits for your selected URLs. You can crawl website pages using either the AWS](https://console.aws.amazon.com/bedrock/home) [Management Console for Amazon Bedrock or the CreateDataSource API (see Amazon Bedrock](https://console.aws.amazon.com/bedrock/home) [supp

In [48]:
# 질문의 주제 선택 
question = "Bedrock에서 Converse API를 활용할 때 응답 양식"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

print("==============")
print(context.contents)
print("==============")
status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [2672]
Traversing 'Code examples for Amazon Bedrock using AWS SDKs'...
target_node: [2728]
Traversing 'Code examples for Amazon Bedrock Runtime using AWS SDKs'...
target_node: [3004]
Traversing 'Cohere Command for Amazon Bedrock Runtime using AWS SDKs'...
target_node: [3018]
Traversing 'Invoke Cohere Command on Amazon Bedrock using Bedrock's Converse API with a response stream'...
No child. Proceed to 'get_contents'...
Num Documents: 9
Search Type: get_short_documents
The following code examples show how to send a text message to Cohere Command, using Bedrock's Converse API and process the response stream in real-time. .NET AWS SDK for .NET Note There's more on GitHub. Find the complete example and learn how to set up and run [in the AWS Code Examples Repository.](https://github.com/awsdocs/aws-doc-sdk-examples/tree/main/dotnetv3/Bedrock-runtime#code-examples) Sen

In [49]:
# 질문의 주제 선택 (찾을 수 없는 정보)
question = "Bedrock의 가격 정책"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

print("==============")
print(context.contents)
print("==============")
status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [3333]
Traversing 'Quotas for Amazon Bedrock'...
target_node: [3346]
Traversing 'Model inference prompt quotas'...
No child. Proceed to 'get_contents'...
Num Documents: 2
Search Type: get_short_documents
Select a tab to see model-specific quotas for prompts. Amazon Titan Text models |Description|Value|Adjustable through Service Quotas| |---|---|---| |Text prompt length, in characters|42,000|No| Amazon Titan Image Generator G1 V1 |Description|Value|Adjustable through Service Quotas| |---|---|---| |Text prompt length, in characters|1,024|No| Model inference prompt quotas 1574 |Description|Value|Adjustable through Service Quotas| |---|---|---| |Input image size|5 MB|No| |Input image height in pixels (inpainting/outpainting)|1,024|No| |Input image width in pixels (inpainting/outpainting)|1,024|No| |Input image height in pixels (image variation)|4,096|No| |Input image 

In [24]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

searching_scheme = "keyword" # full_text | keyword | vector
csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def subgraph_level_search_dev(question, graph, subgraph, language = "English", k=5):

    if searching_scheme == "vector":
        print("vector search started")

        sys_prompt_template = """
            당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 벡터 검색으로 문서를 찾아내기에 적합한 질문을 만들어주세요.({language})
            
            주의: 

            - 문서 이름과 질문을 고려하여 해당 문서 내에서 가장 관련성 높고 특징적인 질문을 생성하세요.
            - 문서의 특정 내용을 잘 나타내는 자연어 질문을 선호합니다.
        """
        usr_prompt_template = "#검색 대상 문서 이름:\n{subgraph}\n\n#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"

        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, subgraph=subgraph, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
        print(keywords)
        index_name = "content_embedding_index"

        embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name=region_name)
        question_embedding = embeddings.embed_query(keywords)

        vector_search_query = """
        MATCH (root:Title {level: "1", value: $subgraph})
        MATCH (root)-[:HAS_CHILD*0..]->(title:Title)-[:HAS_CONTENTS]->(content:Content)

        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WHERE node = content

        RETURN node.text AS text, score
        ORDER BY score DESC
        LIMIT $k
        """

        params = {
            "subgraph": subgraph,
            "question_embedding": question_embedding,
            "k": k,
            "index_name": index_name
        }

        vector_store = Neo4jVector.from_existing_index(
            embedding=embeddings,
            index_name=index_name,
            node_label="Content",
            text_node_property="text", 
            embedding_node_property="embedding"
        )

        search_results = vector_store.query(vector_search_query, params=params)

    else:
        print("text search started")
        sys_prompt_template = """
        당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 핵심 키워드를 1개 추출합니다.
        키워드는 다음 조건을 반드시 만족해야 합니다:
        1. 키워드에 '_', '-' 등의 특수 문자 포함 금지 (예: custom_model 대신 custom model로 응답)
        2. 주어진 문서 이름 내에서 질문의 맥락에 가장 적합한 단어를 선택 (문서 이름은 키워드에 포함할 필요가 없음)
        2. 문서 이름에 이미 포함된 내용보다는 검색하려는 특정 기능 및 개념을 잘 나타내는 구체적 단어를 선택
        3. {language} 키워드 제공 
        
        주의: 
        - 문서 이름과 질문을 고려하여 해당 문서 내에서 가장 관련성 높고 특징적인 단어를 선택하세요.
        - 너무 일반적인 단어보다는 문서의 특정 내용을 잘 나타내는 단어를 선호합니다.
        """
        usr_prompt_template = "#검색 대상 문서 이름:\n{subgraph}\n\n#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"
        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, subgraph=subgraph, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
    
        
        if searching_scheme == "full_text":
            search_query = """
                MATCH (root:Title {level: "1", value: $subgraph})
                MATCH (root)-[:HAS_CHILD*0..]->(title:Title)-[:HAS_CONTENTS]->(content:Content)

                CALL db.index.fulltext.queryNodes("Search_Content_by_FullText", $keywords) YIELD node, score
                WHERE node = content

                RETURN node.text as text, score, title.name as title_name, title.level as title_level
                ORDER BY score DESC
                LIMIT $k
            """
        elif searching_scheme == "keyword":
            search_query = """
                MATCH (root:Title {level: "1", value: $subgraph})
                MATCH (root)-[:HAS_CHILD*0..]->(title:Title)-[:HAS_CONTENTS]->(content:Content)
                WITH content, title, $keywords AS keyword
                WHERE content.text CONTAINS keyword
                RETURN content.text AS text, 
                    size(split(toLower(content.text), toLower(keyword))) - 1 AS score,
                    {
                        title: title.name,
                        level: title.level,
                        value: title.value
                    } AS metadata
                ORDER BY score DESC
                LIMIT $k
            """
        
        params = { "subgraph": subgraph, "k": k, "keywords": keywords}
        search_results = graph.run(search_query, params)
        
    content = "\n\n\n".join(f"{record['text']} (Score: {record['score']})" for record in search_results)
    return content

In [25]:
# 질문의 주제 선택 (찾을 수 없는 정보)
question = "Bedrock의 가격 정책"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

if status == 'Partial':
    context.contents = get_sibling_contents_dev(graph, context.parent_id, context.contents, context.k)
elif status == 'None':
    context.contents = subgraph_level_search_dev(question, graph, subgraph, 'English', 5)

print(context.contents)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [3333]
Traversing 'Quotas for Amazon Bedrock'...
Num Documents: 3
Search Type: get_short_documents
Your AWS account has default quotas, formerly referred to as limits, for Amazon Bedrock. To view [service quotas for Amazon Bedrock, follow the steps at Viewing service quotas and select Amazon](https://docs.aws.amazon.com/servicequotas/latest/userguide/gs-request-quota.html) Bedrock as the service. Some quotas differ by model. Unless specified otherwise, a quota applies to all versions of a model. To maintain the performance of the service and to ensure appropriate usage of Amazon Bedrock, the default quotas assigned to an account might be updated depending on regional factors, payment history, fraudulent usage, and/or approval of a quota increase request. You can request a quota increase for your account by following the steps below: - If a quota is marked as Yes i

In [26]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

searching_scheme = "vector" # full_text | keyword | vector
csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def global_search_dev(question, graph, language = "English", k=5):

    if searching_scheme == "vector":
        sys_prompt_template = """
            당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 벡터 검색으로 문서를 찾아내기에 적합한 질문을 만들어주세요.({language})
            
            주의: 

            - 질문을 고려하여 가장 관련성 높고 특징적인 질문을 생성하세요.
            - 사용자의 질문 의도를 잘 반영하는 자연어 질문을 선호합니다.
        """
        usr_prompt_template = "#\n\n#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"

        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)

        index_name = "content_embedding_index"

        embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name=region_name)
        question_embedding = embeddings.embed_query(keywords)

        vector_search_query = """
        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WITH DISTINCT node, score
        WHERE node:Content
        RETURN node.text AS text, score
        ORDER BY score DESC
        """

        params = {
            "question_embedding": question_embedding,
            "k": k,
            "index_name": index_name
        }

        vector_store = Neo4jVector.from_existing_index(
            embedding=embeddings,
            index_name=index_name,
            node_label="Content",
            text_node_property="text", 
            embedding_node_property="embedding"
        )

        search_results = vector_store.query(vector_search_query, params=params)

    else:
        sys_prompt_template = """
        당신은 AWS에 정통한 전문 엔지니어입니다. 사용자의 질문을 바탕으로 매뉴얼에서 핵심 키워드를 2개 추출합니다.
        키워드는 다음 조건을 반드시 만족해야 합니다:
        1. 키워드에 '_', '-' 등의 특수 문자 포함 금지 (예: custom_model 대신 custom model로 응답)
        2. 문서 이름에 이미 포함된 내용보다는 검색하려는 특정 기능 및 개념을 잘 나타내는 구체적 단어를 선택
        3. {language} 키워드 제공 
        
        주의: 
        - 너무 일반적인 단어보다는 서비스 이름과 서비스의 특정 내용을 잘 나타내는 단어를 선호합니다.
        """
        usr_prompt_template = "#질문:\n{question}\n\n #응답 형식:\n{csv_list_response_format}"
        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, language=language, question=question, csv_list_response_format=csv_list_response_format)
        model_id = low_level_model 
        keywords = converse_with_bedrock(sys_prompt, usr_prompt, model_id)
        print(keywords)
        
        if searching_scheme == "full_text":
            search_query ="""
            CALL db.index.fulltext.queryNodes("Search_Content_by_FullText", $keywords) YIELD node, score
            WHERE node:Content
            OPTIONAL MATCH (title:Title)-[:HAS_CONTENTS]->(node)
            RETURN node.text as text, score, title.name as title_name, title.level as title_level
            ORDER BY score DESC
            LIMIT $k
            """
        elif searching_scheme == "keyword":
            search_query = """
            MATCH (content:Content)
            WITH content, $keywords AS keyword
            WHERE content.text CONTAINS keyword
            OPTIONAL MATCH (title:Title)-[:HAS_CONTENTS]->(content)
            RETURN content.text AS text, 
                size(split(toLower(content.text), toLower(keyword))) - 1 AS score,
                {
                    title: title.name,
                    level: title.level,
                    value: title.value
                } AS metadata
            ORDER BY score DESC
            LIMIT $k
            """
        
        params = {"k": k, "keywords": keywords}
        search_results = graph.run(search_query, params)
        
    content = "\n\n\n".join(f"{record['text']} (Score: {record['score']})" for record in search_results)
    return content

In [27]:
# 질문의 주제 선택 (찾을 수 없는 정보)
question = "Amazon Bedrock의 가격 정책"

content = global_search_dev(question, graph, 'English', 5)
print(content)

/home/ubuntu/.local/lib/python3.10/site-packages/langchain_community/vectorstores/neo4j_vector.py:531: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


Topics - Features of Amazon Bedrock - Amazon Bedrock pricing - Supported AWS Regions - Key definitions (Score: 0.8607656359672546)


Charges for Guardrails for Amazon Bedrock will be incurred only for the policies configured in the
[guardrail. The price for each policy type is available at Amazon Bedrock Pricing. If guardrails blocks](https://aws.amazon.com/bedrock/pricing/)
the input prompt, you will be charged for the guardrail evaluation. There will be no charges for
foundation model inference calls. If guardrails blocks the model response, you will be charged for
guardrails evaluation of the input prompt and the model response. In this case, you will be charged
for the foundation model inference calls as well the model response that was generated prior to
guardrails evaluation.  
How charges are calculated 358 (Score: 0.8290265798568726)


What is Amazon Bedrock? 1 Features of Amazon Bedrock .. 1 Amazon Bedrock pricing 2 Supported AWS Regions 2 Key definitions ... 5 Basic concepts 

In [38]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def generate_answer_dev(question, context):
    # Prompt setting
    sys_prompt_template = "당신은 AWS에 정통한 전문 엔지니어입니다. 주어진 사전 정보만 활용하여, 사용자 질문에 답변을 생성하세요. 사전 정보로 주어지지 않은 내용에 대한 질문에는 모른다고 답변하세요."
    usr_prompt_template = "#사전 정보: {context}\n\n #사용자 질문:\n {question}"
    prompt = ChatPromptTemplate.from_messages([("system", sys_prompt_template), ("human",usr_prompt_template)])

    # Model setting
    model_kwargs = {
            "temperature": 0.5,
            "max_tokens": 4096
        }
    llm = ChatBedrock(model_id=high_level_model, region_name="us-west-2", model_kwargs=model_kwargs, streaming=True)   

    # Output setting
    parser = StrOutputParser()

    # Chain
    chain = prompt | llm | parser
    for chunk in chain.stream({"context": context, "question": question}):
        print(chunk, end="", flush=True)

In [42]:
# 질문의 주제 선택 
question = "Bedrock에서 Agent의 성능을 최적화하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

if status == 'Partial':
    context.contents = get_sibling_contents_dev(graph, context.parent_id, context.contents, context.k)
elif status == 'None':
    context.contents = subgraph_level_search_dev(question, graph, subgraph, 'English', 5)

generate_answer_dev(question, context.contents)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1359]
Traversing 'Agents for Amazon Bedrock'...
target_node: [1521]
Traversing 'Manage an Amazon Bedrock agent'...
target_node: [1565]
Traversing 'Manage agent memory'...
target_node: [1575]
Traversing 'Disable memory for your Amazon Bedrock agent'...
No child. Proceed to 'get_contents'...
Num Documents: 2
Search Type: get_short_documents
None
vector search started
Bedrock에서 Agent의 성능을 최적화하는 방법:

agent memory size, agent batch size, agent learning rate, agent reward shaping, agent exploration strategy


/home/ubuntu/.local/lib/python3.10/site-packages/langchain_community/vectorstores/neo4j_vector.py:531: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


사전 정보에 따르면, Agents for Amazon Bedrock에는 특정 사용 사례에 대한 성능 최적화 옵션이 제공됩니다. 단일 지식베이스를 사용하는 간단한 사용 사례의 경우, 레이턴시 최적화를 위한 다양한 흐름을 선택할 수 있습니다. 

성능 최적화를 위해서는 다음과 같은 조건이 충족되어야 합니다:

1. 에이전트에 단일 지식베이스만 포함되어 있어야 합니다.
2. 에이전트에 액션 그룹이 없거나 모두 비활성화되어 있어야 합니다.
3. 에이전트가 사용자로부터 추가 정보를 요청하지 않아야 합니다.
4. 에이전트가 기본 오케스트레이션 프롬프트 템플릿을 사용해야 합니다.

이러한 조건을 충족하는지 확인하는 방법은 콘솔 또는 API를 통해 에이전트를 관리하는 방법에 따라 다릅니다. 콘솔에서는 에이전트 페이지에서 이 정보를 확인할 수 있으며, API를 통해서는 CreateAgent 요청에서 관련 필드를 확인할 수 있습니다.

In [43]:
# 질문의 주제 선택
question = "Bedrock에서 Knowledge Base를 테스트하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

if status == 'Partial':
    context.contents = get_sibling_contents_dev(graph, context.parent_id, context.contents, context.k)
elif status == 'None':
    context.contents = subgraph_level_search_dev(question, graph, subgraph, 'English', 5)

generate_answer_dev(question, context.contents)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1070]
Traversing 'Knowledge bases for Amazon Bedrock'...
target_node: [1284]
Traversing 'Test a knowledge base in Amazon Bedrock'...
target_node: [1286]
Traversing 'Query the knowledge base and return results or generate responses'...
No child. Proceed to 'get_contents'...
Num Documents: 9
Search Type: get_short_documents
Partial
Bedrock에서 Knowledge Base를 테스트하는 방법은 다음과 같습니다:

1. AWS Management Console에 로그인하고 Bedrock 콘솔(https://console.aws.amazon.com/bedrock/)로 이동합니다.
2. 왼쪽 탐색 창에서 Knowledge bases를 선택합니다.
3. Knowledge bases 섹션에서 다음 중 하나를 수행합니다:
   - 테스트하려는 Knowledge Base 옆의 라디오 버튼을 선택하고 Test knowledge base를 클릭합니다. 오른쪽에서 테스트 창이 열립니다.
   - 테스트하려는 Knowledge Base를 선택합니다. 오른쪽에서 테스트 창이 열립니다.
4. Generate responses for your query 옵션을 선택하거나 선택 해제합니다:
   - 지식 베이스에서 직접 검색된 정보를 반환하려면 Generate responses를 끕니다. Bedrock은 관련 텍스트 청크를 반환합니다.
   - 지식 베이스에서 검색된 정보를 기반으로 응답을 생성하려면 Gener

In [44]:
# 질문의 주제 선택 (문서에 없는 정보)
question = "Bedrock의 가격 정책"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

if status == 'Partial':
    context.contents = get_sibling_contents_dev(graph, context.parent_id, context.contents, context.k)
elif status == 'None':
    context.contents = subgraph_level_search_dev(question, graph, subgraph, 'English', 5)

generate_answer_dev(question, context.contents)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [3333]
Traversing 'Quotas for Amazon Bedrock'...
target_node: [3346]
Traversing 'Model inference prompt quotas'...
No child. Proceed to 'get_contents'...
Num Documents: 2
Search Type: get_short_documents
None
vector search started
Bedrock의 가격 정책에 대한 질문:

on-demand pricing, pay-as-you-go, no upfront costs, pricing based on usage


/home/ubuntu/.local/lib/python3.10/site-packages/langchain_community/vectorstores/neo4j_vector.py:531: ExperimentalWarning: The configuration may change in the future.
  self._driver.verify_connectivity()


사전 정보에 따르면 Amazon Bedrock의 가격 정책은 다음과 같습니다:

1. 기본 가격 정책:
   - Amazon Bedrock 사용 시 입력 토큰과 출력 토큰의 볼륨에 따라 과금됩니다.
   - 모델 제공업체별로 가격이 다르게 책정되어 있습니다.

2. 프로비저닝된 처리량(Provisioned Throughput) 옵션:
   - 프로비저닝된 처리량을 구매하면 더 저렴한 가격으로 사용할 수 있습니다.
   - 프로비저닝된 처리량에 대한 자세한 내용은 "Provisioned Throughput for Amazon Bedrock" 문서를 참고하세요.

3. Guardrails 사용 시 과금:
   - Guardrails 정책 구성에 따라 과금이 발생합니다.
   - Guardrails에서 입력 프롬프트를 차단하는 경우 Guardrails 평가 비용이 청구됩니다.
   - Guardrails에서 모델 응답을 차단하는 경우 Guardrails 평가 비용과 모델 추론 비용이 청구됩니다.

4. AWS 계정 및 청구:
   - AWS 계정에 자동으로 모든 AWS 서비스가 등록되지만, 실제로 사용한 서비스에 대해서만 과금됩니다.
   - AWS 청구 및 계정 관련 문의는 AWS 지원팀에 문의하시기 바랍니다.

In [45]:
# 질문의 주제 선택
question = "What is Amazon Bedrock Playground?"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node:
    print("target_node:", target_node)
    result = traverse_child_dev(question, subgraph, graph, target_node)
    traverse_results.append(result)

    if result.next_action == "get_content":
        break

    if result.selected_child_ids:
        target_node = [result.selected_child_ids[0]]
    else:
        break

# 문서 얻어내기
context = get_contents_dev(graph, result.parent_id, 5)
print("==============")
print("Search Type:", context.search_type)
if context.search_type == 'node_level_search':
    context.contents = node_level_search_dev(question, graph, context.parent_id, context.parent_name, context.k)

status = check_relevance_dev(question, context.contents, context.parent_name, context.contents_length, context.search_type, context.k)
print(status)

if status == 'Partial':
    context.contents = get_sibling_contents_dev(graph, context.parent_id, context.contents, context.k)
elif status == 'None':
    context.contents = subgraph_level_search_dev(question, graph, subgraph, 'English', 5)

generate_answer_dev(question, context.contents)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [19]
Traversing 'What is Amazon Bedrock?'...
target_node: [22]
Traversing 'Features of Amazon Bedrock'...
No child. Proceed to 'get_contents'...
Num Documents: 2
Search Type: get_short_documents
Complete
Amazon Bedrock Playground is a feature of the Amazon Bedrock service that allows you to experiment with prompts and configurations for foundation models. The Bedrock Playground provides a graphical interface in the AWS console where you can:

- Run model inference by sending prompts to different foundation models and see the generated responses.
- Explore how to use the API to make requests to the InvokeModel APIs for your application.
- Experiment with different prompts and configurations to understand the capabilities of the foundation models.

The Bedrock Playground lets you test and explore the features of Amazon Bedrock without having to set up your own appli